In [1]:
import git
import sys

repo = git.Repo(search_parent_directories=True)
sys.path.append(repo.working_dir)

from helper_tools import parser

triple_df, entity_df, docs = parser.synthie_parser("train_text", 10)

2025-06-02 11:22:19,505 - INFO - Initializing llama-3.3-70b at Cerebras - 30.0 RPM
2025-06-02 11:22:19,522 - INFO - Embeddings model bge-m3 initialized
2025-06-02 11:22:19,530 - INFO - Langfuse handler initialized
2025-06-02 11:22:19,538 - INFO - Langfuse client initialized
2025-06-02 11:22:19,539 - INFO - Initializing Qdrant vector store
2025-06-02 11:22:21,543 - INFO - Qdrant vector store initialized
2025-06-02 11:22:21,615 - INFO - Qdrant description vector store initialized
2025-06-02 11:22:21,683 - INFO - Qdrant example vector store initialized
2025-06-02 11:22:21,976 - INFO - Dataset wikidata_predicates: exists, triple count: 53680
2025-06-02 11:22:22,633 - INFO - Dataset wikidata_class_hierarchy: exists, triple count: 1612566
2025-06-02 11:22:22,634 - INFO - Fuseki clients initialized
2025-06-02 11:22:22,635 - INFO - SPARQL wrapper initialized


Fetching 27 files:   0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 92794.34it/s]


Preparing entities for bulk upload to qdrant...


Preparing entities: 100%|██████████| 46/46 [00:01<00:00, 37.43it/s]


No new entities to upload. 46 entities were already in the database.
Preparing predicates for bulk upload to qdrant...


Preparing predicates: 100%|██████████| 26/26 [00:00<00:00, 38.71it/s]

No new predicates to upload. 26 predicates were already in the database.


In [2]:
import os
import dspy
import git
from dotenv import load_dotenv

repo = git.Repo(search_parent_directories=True).working_dir
load_dotenv(repo + ".env")

lm = dspy.LM('openai/Meta-Llama-3.3-70B-Instruct', api_key=os.getenv("SAMBANOVA_API_KEY"), api_base='https://api.sambanova.ai/v1')
dspy.configure(lm=lm)

In [3]:
from typing import List
import traceback
from helper_tools.base_setup import label_vector_store, example_vector_store
import re
from qdrant_client.http import models

def uri_search_tool(search_terms: List[str]):
    """Fetch URIs for search terms from the underlying knowledge graph. Each Search Term must be given as a string followed by one of the search modes in [] brackets:
    [Q] - Entity Search
    [P] - Property Search by Term
    [X] - Property Search by Example Sentence

    For example: {search_terms: [Angela Merkel[Q], CDU[Q], member of[P], Angela Merkel is member of CDU[X]]}
    """
    try:
        search_response = ""

        for term in search_terms:
            # Parse search term and mode
            mode = None
            clean_term = term

            # Extract mode from term using regex
            if "[" in term and "]" in term:
                mode_match = re.search(r'\[([QPX])\]', term)
                if mode_match:
                    mode = mode_match.group(1)
                    clean_term = re.sub(r'\[([QPX])\]', '', term).strip()

            # Default behavior if no mode is specified
            if not mode:
                # Assume it's a label search
                results = label_vector_store.similarity_search_with_score(clean_term, k=3)
                search_response += f'Most Similar Search Results for "{clean_term}" - Default Search Mode (LABEL):\n'

                for idx, (doc, sim_score) in enumerate(results):
                    search_response += f"  {idx+1}. Label: {doc.page_content}\n"
                    search_response += f"     URI: {doc.metadata['uri']}\n"
                    search_response += f"     Description: {doc.metadata['description']}\n"
                search_response += "\n"
                continue

            # Create filter condition if mode is Q or P
            filter_condition = None
            if mode in ['Q', 'P']:
                filter_condition = models.Filter(
                    must=[
                        models.FieldCondition(
                            key="metadata.type",
                            match=models.MatchValue(value="entity" if mode == "Q" else "predicate")
                        )
                    ]
                )

            # Perform search based on the mode
            if mode in ['Q', 'P']:
                # Search in labels with type filter
                results = label_vector_store.similarity_search_with_score(
                    clean_term,
                    k=3,
                    filter=filter_condition
                )
                search_response += f'Similar Search Results for "{clean_term}" - Search Mode [{mode}]:\n'

                for idx, (doc, sim_score) in enumerate(results):
                    search_response += f"  - Label: {doc.page_content}\n"
                    search_response += f"     URI: {doc.metadata['uri']}\n"
                    search_response += f"     Description: {doc.metadata['description']}\n"
                    if doc.metadata.get("example"):
                        search_response += f"     Example: {doc.metadata['example']}\n"

            elif mode == 'X':
                # Search in examples collection
                results = example_vector_store.similarity_search_with_score(
                    clean_term,
                    k=3
                )
                search_response += f'Similar Search Results for "{clean_term}" - Search Mode [X]:\n'

                for idx, (doc, sim_score) in enumerate(results):
                    search_response += f"  - Label: {doc.metadata['label']}\n"
                    search_response += f"     URI: {doc.metadata['uri']}\n"
                    search_response += f"     Description: {doc.metadata['description']}\n"
                    search_response += f"     Example: {doc.page_content}\n"

            search_response += "\n"
        return search_response

    except Exception as e:
        return f"SYSTEM MESSAGE: Error occurred in URI search tool: {str(e)}\n{traceback.format_exc()}"

In [4]:
import dspy

class cIEAgent(dspy.Signature):
    """You are an expert for closed information extraction (the extraction of triples using a given text, where the triples match an underlying knowledge graph.). You will be given a text and should extract triples out of it and map those triples onto the underlying knowledge graph.

    You are given a list of tools to handle user request, and you should decide the right tool to use in order to fullfil users' request.
    """

    text: str = dspy.InputField()
    turtle_string: str = dspy.OutputField(
        desc=(
                "Output of the extracted triples from the text. Format: Turtle. Restricted to wd: namespace. Usage of wdt: is prohibited."
            )
        )

In [5]:
agent = dspy.ReAct(
    cIEAgent,
    tools = [
        uri_search_tool
    ]
)

In [6]:
result = agent(text=docs.iloc[0]["text"])

In [7]:
result.turtle_string

'wd:Q18020434 wdt:P1366 wd:Q50839256 .\nwd:Q50839256 wdt:P749 wd:Q59261 .\nwd:Q50839256 wdt:P159 wd:Q393454 .'

In [8]:
# noinspection PyTypeChecker
for i in range(len([key for key in result.trajectory.keys() if key.startswith('thought_')])):
    thought = result.trajectory.get(f'thought_{i}', '')
    tool_name = result.trajectory.get(f'tool_name_{i}', '')
    tool_args = result.trajectory.get(f'tool_args_{i}', '')
    observation = result.trajectory.get(f'observation_{i}', '')

    print(f"Thought {i}: {thought}\n")
    print(f"Tool Name {i}: {tool_name}")
    print(f"Tool Args {i}: {tool_args}")
    print(f"Tool Output {i}: {observation}\n{'-'*40}")


Thought 0: To extract triples from the given text and map them onto the underlying knowledge graph, we first need to identify key entities and their relationships. The text mentions the "General Administration of Quality Supervision, Inspection and Quarantine," the "State Administration for Market Regulation," and the "State Council of the People's Republic of China." It also mentions the location of the headquarters, "Haidian District, China." Our next step should be to find URIs for these entities in the knowledge graph.

Tool Name 0: uri_search_tool
Tool Args 0: {'search_terms': ['General Administration of Quality Supervision, Inspection and Quarantine[Q]', 'State Administration for Market Regulation[Q]', "State Council of the People's Republic of China[Q]", 'Haidian District, China[Q]']}
Tool Output 0: Similar Search Results for "General Administration of Quality Supervision, Inspection and Quarantine" - Search Mode [Q]:
  - Label: General_Administration_of_Quality_Supervision,_Ins

In [19]:
dataset = []

# Gruppiere triples nach docid
grouped_triples = triple_df.groupby("docid")

for i, doc in docs.iterrows():
    docid = doc["docid"]
    text = doc["text"]

    # Liste der triples für diesen docid
    turtle_triples = []

    if docid in grouped_triples.groups:
        group = grouped_triples.get_group(docid)

        for _, row in group.iterrows():
            subject_uri = row["subject_uri"].strip()
            predicate_uri = row["predicate_uri"].strip()
            object_uri = row["object_uri"].strip()
            turtle_triples.append(f"<{subject_uri}> <{predicate_uri}> <{object_uri}> .")

    # Alle Tripel zusammenfügen zu einem Turtle-String
    turtle_string = "\n".join(turtle_triples)

    # Beispiel erstellen
    dataset.append(dspy.Example(text=text, turtle_string=turtle_string).with_inputs("text"))

trainset = dataset


In [20]:
from helper_tools.evaluation import evaluate_doc, calculate_scores_from_array, compare_turtle_strings


def scoring(example, pred, trace=None):
    values_array = compare_turtle_strings(predicted_turtle_string=pred.turtle_string, ground_truth_turtle_string=example.turtle_string)
    return calculate_scores_from_array(values_array).loc["Triple"]["F1-Score"]

In [21]:
tp = dspy.MIPROv2(metric=scoring, auto="light", num_threads=1)
optimized_agent = tp.compile(agent, trainset=trainset, requires_permission_to_run=False)

2025/06/02 11:28:02 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 8

2025/06/02 11:28:02 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/02 11:28:02 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/02 11:28:02 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  0%|          | 0/2 [00:00<?, ?it/s]11:28:02 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
2025-06-02 11:28:02,884 - INFO - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
/Users/i538914/Documents/Uni/Masterarbeit/CIExMAS/.pyvenv311/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1748856484.3721714, input_type=float])
  return self.__pydantic_serializer__.to_python(
11:28:04 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
2025-06-02 11:28:04,539 - INFO - Wrapper: Completed Call, calling success_handler
11:28:04 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
2025-06-02 11:28:04,539 - INFO - selected model name for cost calculation: openai/Meta

Bootstrapped 1 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/6


 50%|█████     | 1/2 [00:01<00:01,  1.59s/it]


...

11:38:11 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
2025-06-02 11:38:11,029 - INFO - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
/Users/i538914/Documents/Uni/Masterarbeit/CIExMAS/.pyvenv311/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1748857092.1011713, input_type=float])
  return self.__pydantic_serializer__.to_python(
11:38:12 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
2025-06-02 11:38:12,192 - INFO - Wrapper: Completed Call, calling success_handler
11:38:12 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
2025-06-02 11:38:12,194 - INFO - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
11:38:12 - Li

Average Metric: 2.68 / 7 (38.3%):  88%|████████▊ | 7/8 [00:20<00:02,  2.93s/it]

11:38:13 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
2025-06-02 11:38:13,119 - INFO - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
/Users/i538914/Documents/Uni/Masterarbeit/CIExMAS/.pyvenv311/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1748857094.103575, input_type=float])
  return self.__pydantic_serializer__.to_python(
11:38:14 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
2025-06-02 11:38:14,195 - INFO - Wrapper: Completed Call, calling success_handler
11:38:14 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
2025-06-02 11:38:14,196 - INFO - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
11:38:14 - Lit

Average Metric: 3.35 / 8 (41.9%): 100%|██████████| 8/8 [00:22<00:00,  2.84s/it]

2025/06/02 11:38:14 INFO dspy.evaluate.evaluate: Average Metric: 3.3490842490842487 / 8 (41.9%)
2025/06/02 11:38:14 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 41.86 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 0', 'Predictor 1: Few-Shot Set 5'].
2025/06/02 11:38:14 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [7.64, 6.25, 41.42, 7.14, 36.51, 35.26, 18.95, 38.09, 0.0, 19.23, 29.62, 41.42, 41.42, 42.31, 42.31, 42.31, 42.31, 36.96, 42.31, 32.68, 41.86]
2025/06/02 11:38:14 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 42.31
2025/06/02 11:38:14 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/06/02 11:38:14 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 42.31!


12:48:34 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
2025-06-02 12:48:34,019 - INFO - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
/Users/i538914/Documents/Uni/Masterarbeit/CIExMAS/.pyvenv311/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1748861316.237576, input_type=float])
  return self.__pydantic_serializer__.to_python(
12:48:36 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
2025-06-02 12:48:36,273 - INFO - Wrapper: Completed Call, calling success_handler
12:48:36 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
2025-06-02 12:48:36,274 - INFO - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
12:48:36 - Lit

Prediction(
    trajectory={'thought_0': 'To extract triples from the given text and map them onto the underlying knowledge graph, we first need to identify key entities and their relationships. The text mentions "The General Administration of Quality Supervision, Inspection and Quarantine," "the State Administration for Market Regulation," and "the State Council of the People\'s Republic of China" as entities. It also mentions the relationship between these entities, such as one being replaced by another and their hierarchical structure. We need to find URIs for these entities and possibly for the relationship types (e.g., "replaced by," "parent organization") in the knowledge graph.', 'tool_name_0': 'uri_search_tool', 'tool_args_0': {'search_terms': ['General Administration of Quality Supervision, Inspection and Quarantine', 'State Administration for Market Regulation', "State Council of the People's Republic of China", 'replaced by', 'parent organization']}, 'observation_0': 'Most S

In [27]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm

evaluation_log = []

for i in tqdm(range(5)):
    doc_id = i
    text = trainset[i].text
    trace_id = "NONE"
    try:
        # noinspection PyTypeChecker
        response = optimized_agent(text=text)
        turtle_string = response.turtle_string
    except Exception as e:
        turtle_string = e
        response = {"turtle_string":e}

    evaluation_log.append([doc_id, *compare_turtle_strings(predicted_turtle_string=turtle_string, ground_truth_turtle_string=trainset[i].turtle_string), response["turtle_string"], trace_id])

evaluation_log_df = pd.DataFrame(
    evaluation_log,
    columns=[
        "Doc ID",
        "Correct Triples", "Correct Triples with Parents", "Correct Triples with Related", "Gold Standard Triples", "Total Triples Predicted",
        "Extracted Subjects", "Gold Standard Subjects", "Correct Extracted Subjects",
        "Extracted Predicates", "Gold Standard Predicates", "Correct Extracted Predicates",
        "Detected Predicates Doc Parent", "Detected Predicates Doc Related",
        "Correct Pred Predicates Parents", "Correct Pred Predicates Related",
        "Extracted Objects", "Gold Standard Objects", "Correct Extracted Objects",
        "Extracted Entities", "Gold Standard Entities", "Correct Extracted Entities", "Result String", "Langfuse Trace ID"
    ]
)



 20%|██        | 1/5 [00:01<00:05,  1.28s/it]13:13:10 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
2025-06-02 13:13:10,392 - INFO - 
LiteLLM completion() model= Meta-Llama-3.3-70B-Instruct; provider = openai
/Users/i538914/Documents/Uni/Masterarbeit/CIExMAS/.pyvenv311/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1748862791.9693274, input_type=float])
  return self.__pydantic_serializer__.to_python(
13:13:12 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
2025-06-02 13:13:12,133 - INFO - Wrapper: Completed Call, calling success_handler
13:13:12 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Meta-Llama-3.3-70B-Instruct
2025-06-02 13:13:12,134 - INFO - selected model name for cost calculation: ope

AttributeError: 'str' object has no attribute 'working_dir'

In [30]:
repo = git.Repo(search_parent_directories=True)

excel_file_path = f"{repo.working_dir}/approaches/evaluation_logs/One_Agent/train_text-5-evaluation_log-{os.getenv('LLM_MODEL_PROVIDER')}_{os.getenv('LLM_MODEL_ID').replace('/','-')}-{datetime.now().strftime('%Y-%m-%d-%H%M')}.xlsx"
evaluation_log_df.to_excel(excel_file_path, index=False)


import json
import os.path


log_notes_path = f"{repo.working_dir}/approaches/evaluation_logs/log_notes.json"
try:
    with open(log_notes_path, "r") as log_file:
        log_notes = json.load(log_file)
except FileNotFoundError:
    log_notes = {}

# Use just the filename, not the full path
excel_file_name = os.path.basename(excel_file_path)
log_notes[excel_file_name] = {"short_description": "Optimized with DSPy"}

with open(log_notes_path, "w") as log_file:
    json.dump(log_notes, log_file, indent=4)

In [34]:
optimized_agent.save(f"{repo.working_dir}/approaches/One_Agent/DSPy Testing/optimized_agent.json", save_program=False)